June: Order of process for new resolution.

In [1]:
import netCDF4 as nc

# 1. Realistic Bathymetry

[final_bathymetry.ipynb](http://nbviewer.jupyter.org/urls/bitbucket.org/CanyonsUBC/mackenzie_canyon/raw/tip/bathymetry/notebooks/final_bathymetry.ipynb)

* changed x_final and y_final (number of grid cells on x and y)

In [ ]:
real_bathy_file = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/bathymetry/NEMO_files/realistic/real_bathy_quad.nc')

# 2. Idealized Bathymetry

[final_bathymetry.ipynb](http://nbviewer.jupyter.org/urls/bitbucket.org/CanyonsUBC/mackenzie_canyon/raw/tip/bathymetry/notebooks/final_bathymetry.ipynb)

* changed x_final and y_final (number of grid cells on x and y)

In [ ]:
ideal_bathy_file = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/bathymetry/NEMO_files/idealized/ideal_bathy_quad.nc')

# 3. Raw Grid

[final_bathymetry.ipynb](http://nbviewer.jupyter.org/urls/bitbucket.org/CanyonsUBC/mackenzie_canyon/raw/tip/bathymetry/notebooks/final_bathymetry.ipynb)

* changed x_final and y_final (number of grid cells on x and y)

In [ ]:
grid_file = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/coordinates/raw_coordinates/grid_quad.nc')

# 4. Coordinates

# 5. Depth Levels

[depth_levels.ipynb](http://nbviewer.jupyter.org/urls/bitbucket.org/CanyonsUBC/mackenzie_canyon/raw/tip/bathymetry/notebooks/depth_levels.ipynb)

* calculate new jpkdta, ppdzmin, ppkth, ppacr
* run code and create mesh_mask.nc

# 6. Temperature and Salinity

[make_stratification.ipynb](http://nbviewer.jupyter.org/urls/bitbucket.org/CanyonsUBC/mackenzie_canyon/raw/tip/conditions/notebooks/make_stratification.ipynb)

* changed path_bath and path_mesh

In [ ]:
temp_file = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/conditions/NEMO_files/temperature/temperature_quad_90.nc')

In [ ]:
sal_file = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/conditions/NEMO_files/salinity/salinity_quad_80.nc')

# 7. SSH + Barotropic Velocities and Baroclinic Velocities

[ekman_velocities_boundary.ipynb](http://nbviewer.jupyter.org/urls/bitbucket.org/CanyonsUBC/mackenzie_canyon/raw/tip/conditions/notebooks/ekman_velocities_boundary.ipynb)

* changed bathy_file, grid_g, in_s, in_t, mesh_mask

In [ ]:
tropic_file = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/conditions/NEMO_files/bdy/bdy2d_quad_90_m01.nc')

In [ ]:
clinic_file = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/conditions/NEMO_files/bdy/bdy3d_quad_80_m01.nc')

# 8. Run

In [7]:
cells = 237*177*80
mins = (11*60)+1

min_per_cell = mins/cells

new_cells = 480*360*90
new_mins = min_per_cell * new_cells

new_mins/60

51.053422012443676